In [1]:
import os
import pandas as pd
import pickle
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600

In [2]:
explainer_map = {'conceptx': 'ConceptX', 'aconceptx': 'AntonymConceptX', 'conceptx_r': 'ConceptX-R', 'conceptx_a': 'ConceptX-A', 'conceptshap': 'ConceptSHAP', 'tokenshap': 'TokenSHAP', 'random': 'Random'}
explainer_order = ["Random", "TokenSHAP", "ConceptSHAP", "ConceptX", 'AntonymConceptX', "ConceptX-R", "ConceptX-A"]
MODEL_NAMES = {"gpt2": "GPT-2", "mistral-7b-it":"Mistral-7B-Instruct", "gemma-2-2b":"Gemma-2-2B", "gemma-3-4b":"Gemma-3-4B", "gpt4o-mini": "GPT-4o-mini", "llama-3-3b": "Llama-3.2-3B"}

In [4]:
save_dir = "/cluster/home/kamara/conceptx"
seed_ = 0

In [39]:
folder_path = os.path.join(save_dir, f"results-no-sample/sentiment")  # Replace with your folder path

# Initialize a list to store the dataframes
df_list = []

# Loop through each file in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        full_path = os.path.join(root, file)
        parts = file.split('_')
        print("parts: ", parts)
        
        df = pd.read_csv(full_path)

        if "batch" in parts[1]:
            dataset_idx = 3
            df["batch"] = parts[2]
        else:
            dataset_idx = 1
        df["dataset"] = parts[dataset_idx]
        df["model"] = parts[dataset_idx + 1]
        df["explainer"] = parts[dataset_idx + 2]
        
        if len(parts) > dataset_idx + 4:
            df["steer_replace"] = parts[dataset_idx + 3]
            seed_idx = dataset_idx + 4
        else:
            df["steer_replace"] = None
            seed_idx = dataset_idx + 3

        df["seed"] = parts[seed_idx].split(".")[0]
        df_list.append(df)

# Concatenate all dataframes in the list
df_all = pd.concat(df_list, ignore_index=True)
# Update 'explainer' column based on 'baseline' conditions
df_all['steer_replace'] = df_all['steer_replace'].fillna("remove")
df_all

parts:  ['sentiment', 'batch', '1', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['sentiment', 'batch', '2', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['sentiment', 'batch', '3', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['sentiment', 'batch', '4', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['sentiment', 'batch', '5', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['sentiment', 'batch', '9', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['sentiment', 'batch', '7', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['sentiment', 'batch', '6', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['sentiment', 'batch', '8', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['sentiment', 'batch', '0', 'sst2', 'mistral-7b-it', 'random', 'antonym', '0.csv']
parts:  ['sentiment', 'batch', '0', 'sst2', 'mistral-7b-it', 'random', '0.csv']
parts:  ['sentiment', 'batch', '1', 'sst2', 'mistral-7b-it', 'random', 'antonym', '0.csv']
parts:  ['sentimen

,id,input,p0,p_highest,p_label,aspect,highest_token,label,batch,dataset,model,explainer,steer_replace,seed
0,2207,you ca n't help but feel ` stoked . ',0.340095,0.813690,0.057607,positive,you,stoked,1,sst2,mistral-7b-it,random,remove,0
1,3746,"what an idea , what a thrill ride .",0.971406,0.968208,0.957371,positive,idea,thrill,1,sst2,mistral-7b-it,random,remove,0
2,707,a satisfyingly unsettling ride,0.172535,0.920316,0.022778,positive,unsettling,satisfyingly,1,sst2,mistral-7b-it,random,remove,0
3,3411,updated dickensian sensibility,0.149373,0.212248,0.163017,positive,updated,sensibility,1,sst2,mistral-7b-it,random,remove,0
4,4616,a movie that harps on media-constructed ` issu...,0.219240,0.600110,0.065218,negative,issues,harps,1,sst2,mistral-7b-it,random,remove,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68722,856,point at things that explode into flame,0.611446,0.324381,0.324381,negative,explode,explode,9,sst2,gemma-3-4b,conceptx-a,remove,2
68723,3969,be smarter than any 50 other filmmakers still,0.163956,0.057860,0.057860,negative,smarter,smarter,9,sst2,gemma-3-4b,conceptx-a,remove,2
68724,3595,uncomfortably close to losing my lunch,0.861090,0.152120,0.152120,negative,losing,losing,9,sst2,gemma-3-4b,conceptx-a,remove,2
68725,105,", this gender-bending comedy is generally quit...",0.928238,0.846912,0.126962,positive,bending,funny,9,sst2,gemma-3-4b,conceptx-a,remove,2


In [40]:
### Size of dataset for model
model_list = ["llama-3-3b", "gemma-3-4b", "gpt4o-mini", "mistral-7b-it"]
df = df_all[df_all["model"].isin(model_list)]
df_stats = df.groupby(["dataset", "model", "seed", "explainer", "steer_replace"])["id"].nunique().reset_index()

In [41]:
dataset = "sst2"
model_name = "gemma-3-4b"
df_stats[df_stats['dataset']==dataset]

,dataset,model,seed,explainer,steer_replace,id
0,sst2,gemma-3-4b,0,aconceptx,antonym,938
1,sst2,gemma-3-4b,0,aconceptx,remove,938
2,sst2,gemma-3-4b,0,conceptshap,antonym,963
3,sst2,gemma-3-4b,0,conceptshap,remove,963
4,sst2,gemma-3-4b,0,conceptx,antonym,926
...,...,...,...,...,...,...
67,sst2,mistral-7b-it,2,conceptx-a,remove,924
68,sst2,mistral-7b-it,2,random,antonym,1000
69,sst2,mistral-7b-it,2,random,remove,1000
70,sst2,mistral-7b-it,2,tokenshap,antonym,1000


In [42]:
df = df_all[(df_all["model"].isin(model_list))&(df_all['dataset']==dataset)&(df_all['model']==model_name)&(df_all['steer_replace']=="antonym")]
df_stats2 = df.groupby(["dataset", "batch", "model", "seed", "explainer"])["id"].nunique().reset_index()
df_explore = df_stats2[df_stats2['dataset']==dataset]
for i in range(10):
    print(df_explore[df_explore["batch"]==str(i)])

   dataset batch       model seed    explainer   id
0     sst2     0  gemma-3-4b    0    aconceptx   93
1     sst2     0  gemma-3-4b    0  conceptshap   95
2     sst2     0  gemma-3-4b    0     conceptx   92
3     sst2     0  gemma-3-4b    0   conceptx-a   92
4     sst2     0  gemma-3-4b    0       random  100
5     sst2     0  gemma-3-4b    0    tokenshap  100
6     sst2     0  gemma-3-4b    1    aconceptx   92
7     sst2     0  gemma-3-4b    1  conceptshap   95
8     sst2     0  gemma-3-4b    1     conceptx   91
9     sst2     0  gemma-3-4b    1   conceptx-a   91
10    sst2     0  gemma-3-4b    1       random  100
11    sst2     0  gemma-3-4b    1    tokenshap  100
12    sst2     0  gemma-3-4b    2    aconceptx   93
13    sst2     0  gemma-3-4b    2  conceptshap   98
14    sst2     0  gemma-3-4b    2     conceptx   92
15    sst2     0  gemma-3-4b    2   conceptx-a   92
16    sst2     0  gemma-3-4b    2       random  100
17    sst2     0  gemma-3-4b    2    tokenshap  100
   dataset b

In [46]:
# Convert relevant columns to numeric
df_all["p0"] = pd.to_numeric(df_all["p0"], errors="coerce")
df_all["p_highest"] = pd.to_numeric(df_all["p_highest"], errors="coerce")
df_all["p_label"] = pd.to_numeric(df_all["p_label"], errors="coerce")


In [47]:
# Filter only by dataset and p0 threshold (no longer filtering by model)
df = df_all[(df_all["dataset"] == dataset)&(df_all["model"] == model_name)]
df = df[df["p0"] > 0.5]

# Compute diffs
df["p0_p_highest_diff"] = df["p0"] - df["p_highest"]
df["p0_p_label_diff"] = df["p0"] - df["p_label"]

# Compute stats: group by explainer and model
stats = df.groupby(["explainer", "model", "seed", "steer_replace"])[["p0_p_highest_diff"]].mean().reset_index()#.agg(["mean", "var"])
stats = stats.groupby(["explainer", "model", "steer_replace"])[["p0_p_highest_diff"]].agg(["mean", "var"])
stats

p0_p_highest_diff          
                                                  mean       var
explainer   model      steer_replace                            
aconceptx   gemma-3-4b antonym                0.378157  0.000040
                       remove                 0.296738  0.000003
conceptshap gemma-3-4b antonym                0.353257  0.000054
                       remove                 0.281299  0.000080
conceptx    gemma-3-4b antonym                0.326689  0.000014
                       remove                 0.251912  0.000043
conceptx-a  gemma-3-4b antonym                0.262502  0.000022
                       remove                 0.192841  0.000002
random      gemma-3-4b antonym                0.198869  0.000090
                       remove                 0.132422  0.000142
tokenshap   gemma-3-4b antonym                0.405679  0.000052
                       remove                 0.332557  0.000097

In [48]:
# Compute stats: group by explainer and model
df_label = df[['model', 'explainer', 'seed', 'steer_replace', "p0_p_label_diff"]]
df_label = df_label.rename(columns={'p0_p_label_diff': 'p0_p_highest_diff'})
stats_label = df_label.groupby(["model", "seed", 'steer_replace'])[["p0_p_highest_diff"]].mean()
stats_label = stats_label.groupby(["model", 'steer_replace'])[["p0_p_highest_diff"]].agg(["mean", "var"])
# Step 2: Add new level to index to create the new explainer row
stats_label["explainer"] = "gpt4o-mini_explainer"
stats_label = stats_label.set_index("explainer", append=True)
stats_label.index = stats_label.index.reorder_levels(["explainer", "model", 'steer_replace'])
stats_label

p0_p_highest_diff          
                                                           mean       var
explainer            model      steer_replace                            
gpt4o-mini_explainer gemma-3-4b antonym                0.484220  0.000007
                                remove                 0.416664  0.000014

In [14]:


# Step 3: Concatenate with df1
df_combined = pd.concat([stats, stats_label])

# Step 4: Optional: sort for clarity
df_combined = df_combined.sort_index()


# Flatten multi-index columns
df_combined.columns = [f"{model}_{stat}" for metric, stat in df_combined.columns for model in [metric.split("_diff")[0]]]

# Pivot so each model becomes a column group
df_combined = df_combined.unstack("model")

# Reorder and rename explainers
desired_order = ["random", "tokenshap", "conceptshap", "conceptx", "aconceptx", "conceptx_a", "gpt4o-mini_explainer"]
explainer_map = {
    'conceptx': 'ConceptX',
    'aconceptx': 'AntonymConceptX',
    'conceptx_r': 'ConceptX-R',
    'conceptx_a': 'ConceptX-A',
    'conceptshap': 'ConceptSHAP',
    'tokenshap': 'TokenSHAP',
    'random': 'Random',
    'gpt4o-mini_explainer': 'GPT-4o mini'
}
df_combined = df_combined.rename(index=explainer_map)
df_combined = df_combined.reindex([explainer_map.get(e, e) for e in desired_order])

df_combined.columns = df_combined.columns.swaplevel()
df_combined = df_combined.sort_index(axis=1)
df_combined

model                  gemma-3-4b                      mistral-7b-it  \
                p0_p_highest_mean p0_p_highest_var p0_p_highest_mean   
Random                        NaN              NaN               NaN   
TokenSHAP                     NaN              NaN               NaN   
ConceptSHAP                   NaN              NaN               NaN   
ConceptX                      NaN              NaN               NaN   
AntonymConceptX               NaN              NaN               NaN   
ConceptX-A                    NaN              NaN               NaN   
GPT-4o mini                   NaN              NaN               NaN   

model                             
                p0_p_highest_var  
Random                       NaN  
TokenSHAP                    NaN  
ConceptSHAP                  NaN  
ConceptX                     NaN  
AntonymConceptX              NaN  
ConceptX-A                   NaN  
GPT-4o mini                  NaN

In [24]:

MODEL_NAMES = {"gpt2": "GPT-2", "gemma-2-2b":"Gemma-2-2B", "gemma-3-4b":"Gemma-3-4B", "gpt4o-mini": "GPT-4o mini", "llama-3-3b": "Llama-3.2-3B"}
model_map = MODEL_NAMES
model_order = ["Llama-3.2-3B", "Gemma-3-4B", "GPT-4o mini"]


# Rename models in column MultiIndex
df_combined.columns = pd.MultiIndex.from_tuples([
    (model_map.get(model, model), metric) for model, metric in df_combined.columns
])

# Reorder columns according to model_order
df_combined = df_combined[model_order]
df_combined

Llama-3.2-3B                         Gemma-3-4B  \
                p0_p_highest_mean p0_p_highest_var p0_p_highest_mean   
explainer                                                              
Random                   0.135650         0.066046          0.136672   
TokenSHAP                0.155412         0.071190          0.384658   
ConceptSHAP              0.175997         0.091633          0.321727   
ConceptX                 0.203254         0.098645          0.294545   
AntonymConceptX          0.221914         0.105111          0.386466   
ConceptX-A               0.193819         0.096515          0.236690   
GPT-4o mini              0.500327         0.129132          0.502124   

                                       GPT-4o mini                   
                p0_p_highest_var p0_p_highest_mean p0_p_highest_var  
explainer                                                            
Random                  0.068833          0.137902         0.070212  
TokenSHAP               0.139008          0.428577         0.135817  
ConceptSHAP             0.136756          0.367423         0.139335  
ConceptX                0.130210          0.308379         0.134380  
AntonymConceptX         0.145040          0.346844         0.135727  
ConceptX-A              0.112373          0.300422         0.129533  
GPT-4o mini             0.127368          0.500553         0.127427